## Project overview
Working as an analyst in the SaaS company, we need to regularly compare the product usage data for the previous two months to identify if there's an increase od decrease in client's product uasge.

As a result, I created an automatic data flow to extract product usage data from database as well as clean and add data on the Google Sheet used for looker studio through Google Sheets API, making sure that data is maintained only for the previous two months for comparison.

In [4]:
from google.oauth2.service_account import Credentials 
import gspread
import pyodbc                     
import numpy as np
import pandas as pd
import xlsxwriter
import datetime as dt
from dateutil.relativedelta import relativedelta
from pprint import pprint
from googleapiclient import discovery

##### 1. Since we want to build the dashboard that allows us to compare the performance between previous two months, we need to define the start date and the end date for data extraction from database. 

In [5]:
start_time_yearmonth = (dt.datetime.today() - relativedelta(months=2)).strftime('%Y-%m')
end_time_yearmonth = dt.datetime.today().strftime('%Y-%m') # this month
start_time = '%s-01' % start_time_yearmonth
end_time = '%s-01' % end_time_yearmonth

##### 2. Get the latest clients information from Google Sheet where the information is stored.

In [ ]:
##### 讀帳號資料
scope = ['https://www.googleapis.com/auth/spreadsheets']  # Authorization Scopes
creds = Credentials.from_service_account_file('Your token', scopes=scope)  # Creating google api credentials
gs = gspread.authorize(creds)  # Authentication
sheet_url = 'Your google sheet url'
sheet = gs.open_by_url(sheet_url)  # Open spreadsheet
worksheet = sheet.worksheet('worksheet_name')  # Open worksheet
guest_name_raw = pd.DataFrame(worksheet.get_all_records())
guest_name_raw

##### 3. Join all client's service account name from column into a string, which allows us to quickly use SQL IN function to pull out data at once.

In [ ]:
##### Turn account_name into string for sql in function
acc_txt = ''
for i in range(len(guest_name_raw['account_name'])):
    if i == 0:
        acc_txt += '\'%s\'' % guest_name_raw.loc[i, 'account_name']
    else:
        acc_txt += ',\'%s\'' % guest_name_raw.loc[i, 'account_name']
acc_txt        


##### 4. Log into database using Python and run SQL for different table

In [ ]:
##### Connect to database
conn = pyodbc.connect('Driver={SQL Server};Server=XXX.XX.XX.X;Database=XXX;UID=XX;PWD=XXXXXX')
print('Successfully connect to database')

In [31]:
sql_1 = """SELECT *
            FROM TABLE_1
            WHERE column_A in (%s)  --Enter service accont 
            AND column_B!='XXX'  -- Exclude specific column with specific value. For example, filter out internal value.
            AND time >= '%s' --Start date
            AND time < '%s' --End date
            ORDER by time ASC;  --Sort from oldest to newest """ % (acc_txt, start_time, end_time)



In [ ]:
sql_2 = """SELECT *
            FROM TABLE_2
            WHERE column_A in (%s)  --Enter service accont
            AND time >= '%s'
            AND time < '%s'
            AND status ='success'
            ORDER by time ASC;  --Sort from oldest to newest;""" % (acc_txt, start_time, end_time)

In [35]:
sql_3 = """SELECT a.column_A ,b.* FROM TABLE_3 a inner join TABLE_4 b ON a.column = b.column
           WHERE a.column_A in (%s)
           AND b.time >= '%s'
           AND b.time < '%s'""" % (acc_txt, start_time, end_time)


In [ ]:
print('>>> Extracting search history...', end='')
sys.stdout.flush()
search = pd.read_sql(sql_1, conn)
search.reset_index(drop=True, inplace=True)
print('Done!')

print('>>> Extracting login history...', end='')
sys.stdout.flush()
login = pd.read_sql(sql_2, conn)
print('Done!')

print('>>> Extracting Topic edit history...', end='')
sys.stdout.flush()
changetopic = pd.read_sql(sql_3, conn)
print('Done!')


##### 5. Data cleaning and data processing

In [61]:
sys.stdout.flush()
search = search.merge(log_name, how='inner', left_on='module_name', right_on='English_function_name')   #Replace english function name with Chinese function name
search.drop('English_function_name', axis=1, inplace=True)
search['time'] = pd.to_datetime(search['time'])
print('Done!')

print('>>> Processing login history ...', end='')
sys.stdout.flush()
login.columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6'] #Reorder and select needed columns
print('Done!')
print('Done!')

##### 6. Create one new columns to identify data for last 2 months respectively and the other one to distinguish successful and failed search request 

In [ ]:
# Add column to distinguish data for two different month
search['interval'] = np.where(query['search_time'].dt.month == last_month, 1, 2) 
login['interval'] = np.where(login['time'].dt.month == last_month, 1, 2)
changetopic['interval'] = np.where(changetopic['search_time'].dt.month == last_month, 1, 2)

# Add column to distinguish successful and failed search request
search['successful'] = np.where((search['error_code'] != X) , 'Y', 'N')

print('Data processing finished')

##### 7. Calculate TP90 time based on search query and create new column for data visualization purpose 

In [ ]:
search['search_time'] = search['search_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
search['uresponse_time'] = search['response_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
search['request_time']= (search['response_time']-search['create_time']).dt.total_seconds()
search['searchtime_TP90']=search['request_time'].quantile(0.9)
search = search.fillna('0')

##### 8. Using Google Sheet API to automatically clean and add data on different worksheet on Google Sheet used for looker studio to make sure that the sheet only stores data for two months period

In [11]:
credentials = creds
service = discovery.build('sheets', 'v4', credentials=credentials)

In [68]:
spreadsheet_id = '1JpaEaUmgbE-iis_RieYTnn48kOMsSMri4AFbqnmwAEY'
sheetId = '1099986967'
range_1="first_worksheet!A2:Y"#keep first row(column name)

In [45]:
#Remove data for last month on first worksheet of Googlesheet used for Lookerstudio 
request = service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range=range_)
response = request.execute()

In [ ]:
#Add processed data for this month on first worksheet of Googlesheet used for Lookerstudio 
sheet.worksheet('first_worksheet_name').update('A2:Y',search.values.tolist())

In [47]:
spreadsheet_id = '1JpaEaUmgbE-iis_RieYTnn48kOMsSMri4AFbqnmwAEY'
sheetId = '1321373929'
range_2="second_worksheet!A2:W"

In [48]:
#Remove data for last month on second worksheet on Googlesheet used for Lookerstudio 
request2 = service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range=range_2)
response2 = request2.execute()

In [ ]:
#Add processed data for this month on second worksheet of Googlesheet used for Lookerstudio 
sheet.worksheet('second_worksheet_name').update('A2:G',login.values.tolist())

In [12]:
spreadsheet_id = '1JpaEaUmgbE-iis_RieYTnn48kOMsSMri4AFbqnmwAEY'
sheetId = '1321373929'
range_3="third_worksheet!A2:P"

In [13]:
#Remove data for last month on third worksheet on Googlesheet used for Lookerstudio 
request3 = service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range=range_3)
response3 = request3.execute()

In [ ]:
#Add processed data for this month on third worksheet of Googlesheet used for Lookerstudio 
sheet.worksheet('third_worksheet').update('A2:P',changetopic.values.tolist())